In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pathlib

from re import match
from copy import deepcopy

import gamspec

In [2]:
self = np.linspace(0, 1, 100)
energies = np.linspace(0, 1, 100)
def slice(self, erg: tuple):
    ergstart = erg[0] if erg[0] is not None else energies.min()
    ergstop = erg[1] if erg[1] is not None else energies.max()
    start = np.abs(energies - ergstart).argmin()
    stop = np.abs(energies - ergstop).argmin()
    return self[start:stop+1]

slice(self, (0.93, None))

array([0.92929293, 0.93939394, 0.94949495, 0.95959596, 0.96969697,
       0.97979798, 0.98989899, 1.        ])

In [6]:
pipenet = gamspec.default
files = pathlib.Path(r'E:\Gamut_dev\spectrum_analysis\模拟Eu152能谱\\').glob("Eu152sgl_test_new_*.out")


origins = {}
analyzeds = {}
results_combined = []
for file in files:
    energy = match(r'Eu152sgl_test_new_(\d+\.\d+).out', file.name).group(1)
    energy = float(energy) 

    # if energy < 0.5:
    #     continue

    if pathlib.Path(file.stem+'.xlsx').exists():
        spectrum = gamspec.Spectrum.from_MCNP(str(file))
        df = pd.read_excel(file.stem+'.xlsx')
        analyzed = gamspec.Spectrum.from_xml(file.stem+'.xml')
    else:
        spectrum = gamspec.Spectrum.from_MCNP(str(file))
        spectrum = spectrum.slice((energy/2*1000, None))
        if spectrum.length % 2 != 0:
            spectrum = spectrum[1:]
        analyzed = pipenet([spectrum])[4][0]
        analyzed.export_to_xml(file.stem+'.xml')
        df = analyzed.export_to_pandas()
        df.to_excel(file.stem+'.xlsx')

    aim = df[(df['energy'] - energy*1000).abs()<2]
    aim.loc[:, ['set_energy']] = energy
    origins[energy] = spectrum
    analyzeds[energy] = analyzed
    results_combined.append(aim)


In [7]:
results = pd.concat(results_combined)
results

,Unnamed: 0,left,right,energy,location,height,stderr,area,sig_area2,fitness,set_energy
3,3,152,215,121.961510,208,483479.541148,2.298856,2.785991e+06,2511.320669,0.993895,0.12178
1,1,0,427,244.802526,417,445193.738141,2.644111,2.950656e+06,1972.237285,0.999678,0.24470
2,2,381,598,344.482587,587,323207.465729,2.938387,2.380566e+06,4234.528227,0.999962,0.34428
2,2,614,638,367.936719,627,305253.503186,3.034197,2.321637e+06,1875.404090,0.999232,0.36779
3,3,687,712,411.326864,701,273698.141150,3.114139,2.136485e+06,1794.593411,0.999482,0.41111
2,2,742,769,444.162648,757,253927.122729,3.175014,2.020900e+06,1693.514453,0.999585,0.44398
5,5,1132,1189,688.965151,1174,168297.246990,3.553246,1.498968e+06,2557.160980,0.999987,0.68868
6,6,1313,1345,778.970383,1329,150864.865744,3.701941,1.399934e+06,1381.041789,0.999934,0.77890
2,2,1462,1496,867.509731,1479,137247.476547,3.817933,1.313478e+06,1323.749670,0.999926,0.86739
1,1,1628,1662,964.258026,1645,124774.161092,3.947588,1.234657e+06,1299.600510,0.999939,0.96413


In [ ]:
energies = np.array([40.12, 45.40, 121.78, 244.70, 344.28, 367.79, 411.11, 443.98, 688.68, 778.90, 867.39, 964.13, 1085.91, 1089.70, 1112.12, 1212.95, 1299.12])
percentages = np.array([29.119E-2, 7.304E-2, 14.016E-2, 3.697E-2, 13.128E-2, 0.410E-2, 1.101E-2, 1.372E-2, 0.412E-2, 6.396E-2, 2.048E-2, 7.077E-2, 4.896E-2, 0.844E-2, 6.687E-2, 0.690E-2, 0.802E-2])
def energy_in_group(energy):
    if len(np.where(np.abs(energy-energies)<2)[0]) == 0:
        return None
    else:
        return np.where(np.abs(energy-energies)<2)[0][0]

energy_in_group(411)

6